In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

In [ ]:
import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_set=torchvision.datasets.FashionMNIST(
    root='./data/FashionMNIST',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()])
)

In [ ]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        #convulution layer
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        #linear layer
        self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2=nn.Linear(in_features=120,out_features=60)
        self.out=nn.Linear(in_features=60,out_features=10)


    def forward(self,t):
        #input layer
        t=t
        #conv layer
        t=self.conv1(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        t=self.conv2(t)
        t=F.relu(t)
        t=F.max_pool2d(t,kernel_size=2,stride=2)
        #4th hidden layer
        t=t.reshape(-1,12*4*4)
        t=self.fc1(t)
        t=F.relu(t)
        #5th hidden layer
        t=self.fc2(t)
        t=F.relu(t)
        #6th layer
        t=self.out(t)
        return t


In [ ]:
sample=next(iter(train_set))
image,label=sample

In [ ]:
torch.set_grad_enabled(True)

In [ ]:
net=Network()

In [ ]:
#training loop
params=OrderedDict(
    lr=[0.01,0.001],batch_size=[1000,10000]
)
m=runmanager()
for run in RunBuilder.get_runs(params):

    net=Network()
    loader=DataLoader(train_set,batch_size=run.batch_size,shuffle=True)
    optimizer=optim.Adam(net.parameters(),lr=run.lr)
    m.begin_run(run,net,loader)
    for epoch in range(10):
        m.begin_epoch()
        for batch in loader:
            images=batch[0]
            labels=batch[1]
            preds=net(images)
            loss=F.cross_entropy(preds,labels)#claculate loss
            optimizer.zero_grad() #removing previous grad
            loss.backward() #calculationg grads
            optimizer.step() #updationg weight
            m.track_loss(loss)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('results')
    

In [ ]:
#confusion matrix creation
def get_all_preds(model,loader):
    all_preds=torch.tensor([]) #empty ternsor
    for batch in loader:
        images,labels=batch
        preds=model(images)
        all_preds=torch.cat((all_preds,preds),dim=0)
    return all_preds



In [ ]:
with torch.no_grad():
    prediction_loader=torch.utils.data.DataLoader(train_set,batch_size=10000)
    train_preds=get_all_preds(net,prediction_loader)

In [ ]:
preds_correct=get_num_correct(train_preds,train_set.targets)
print("total correct:",preds_correct)
print("accuracy:",preds_correct/len(train_set))


In [ ]:
confusion_matrix=torch.zeros(10,10,dtype=torch.int32)
confusion_matrix

In [ ]:
for p in stacked:
    tl,pl=p.tolist()
    confusion_matrix[tl,pl]=confusion_matrix[tl,pl]+1

In [ ]:
confusion_matrix

In [ ]:
train_preds

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#tensorboard --logdir runs use this in conda cmd to open tenserboard
tb=SummaryWriter()
images,labels=next(iter(train_loader))
grid=torchvision.utils.make_grid(images)
tb.add_image('images',grid)
tb.add_graph(net,images)
tb.close()

In [ ]:
from collections import OrderedDict,namedtuple
from itertools import product

In [ ]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        run=namedtuple('run',params.keys())
        runs=[]
        for v in product(*params.values()):
            runs.append(run(*v))
        
        return runs




In [ ]:
class runmanager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.net = None
        self.loader = None
        self.tb = None
    def begin_run(self, run, network, loader):

        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.net = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(self.net, images)
    
    def end_run(self):
        self.tb.close()
        self.epoch_count=0

    def begin_epoch(self):
        self.epoch_start_time=time.time()
        self.epoch_count+=1
        self.epoch_loss=0
        self.epoch_num_correct=0

    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        for name, param in self.net.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)


        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)

        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        clear_output(wait=True)
        display(df)


    @torch.no_grad
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
            
    def save(self, fileName):

            pd.DataFrame.from_dict(
                self.run_data, orient='columns'
            ).to_csv(f'{fileName}.csv')

            with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
                json.dump(self.run_data, f, ensure_ascii=False, indent=4)
            
    
    def track_loss(self,loss):
        self.epoch_loss+=loss.item()*self.loader.batch_size

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct+=self._get_num_correct(preds,labels)
   




In [46]:
#cuda implementation

In [56]:
t=torch.ones(1,1,28,28)
net=Network()

In [57]:
t=t.cuda()
net=net.cuda()

In [60]:
gpu=net(t)
gpu.device

device(type='cuda', index=0)

In [61]:
t1=torch.tensor([1,2,3,4])
t2=torch.tensor([1,2,23,4])


In [62]:
t1=t1.to('cuda')
t1.device

device(type='cuda', index=0)

In [63]:
t2=t2.cuda()
t2.device

device(type='cuda', index=0)

In [64]:
t1+t2

tensor([ 2,  4, 26,  8], device='cuda:0')

In [65]:
net

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=10, bias=True)
)

In [66]:
for n,p in net.named_parameters():
    print(p.device,'',n)

cuda:0  conv1.weight
cuda:0  conv1.bias
cuda:0  conv2.weight
cuda:0  conv2.bias
cuda:0  fc1.weight
cuda:0  fc1.bias
cuda:0  fc2.weight
cuda:0  fc2.bias
cuda:0  out.weight
cuda:0  out.bias


In [68]:
torch.cuda.is_available()

True

In [ ]:
#using cuda
class runmanager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None

        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None

        self.net = None
        self.loader = None
        self.tb = None
    def begin_run(self, run, network, loader):

        self.run_start_time = time.time()

        self.run_params = run
        self.run_count += 1

        self.net = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')

        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)

        self.tb.add_image('images', grid)
        self.tb.add_graph(self.net, images.to(getattr(run,'device','cpu'))
        )
    
    def end_run(self):
        self.tb.close()
        self.epoch_count=0

    def begin_epoch(self):
        self.epoch_start_time=time.time()
        self.epoch_count+=1
        self.epoch_loss=0
        self.epoch_num_correct=0

    def end_epoch(self):

        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time

        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)

        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)

        for name, param in self.net.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)


        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results['loss'] = loss
        results["accuracy"] = accuracy
        results['epoch duration'] = epoch_duration
        results['run duration'] = run_duration
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)

        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        clear_output(wait=True)
        display(df)


    @torch.no_grad
    def _get_num_correct(self, preds, labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
            
    def save(self, fileName):

            pd.DataFrame.from_dict(
                self.run_data, orient='columns'
            ).to_csv(f'{fileName}.csv')

            with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
                json.dump(self.run_data, f, ensure_ascii=False, indent=4)
            
    
    def track_loss(self,loss):
        self.epoch_loss+=loss.item()*self.loader.batch_size

    def track_num_correct(self,preds,labels):
        self.epoch_num_correct+=self._get_num_correct(preds,labels)
   


In [ ]:
#training loop
params=OrderedDict(
    lr=[0.01,0.001],batch_size=[1000,10000],device=['cuda','cpu']
)
m=runmanager()
for run in RunBuilder.get_runs(params):
    device=torch.device(run.device)
    net=Network().to(device)
    loader=DataLoader(train_set,batch_size=run.batch_size,shuffle=True)
    optimizer=optim.Adam(net.parameters(),lr=run.lr)
    m.begin_run(run,net,loader)
    for epoch in range(10):
        m.begin_epoch()
        for batch in loader:
            images=batch[0].to(device)
            labels=batch[1].to(device)
            preds=net(images)
            loss=F.cross_entropy(preds,labels)#claculate loss
            optimizer.zero_grad() #removing previous grad
            loss.backward() #calculationg grads
            optimizer.step() #updationg weight
            m.track_loss(loss)
            m.track_num_correct(preds,labels)
        m.end_epoch()
    m.end_run()
m.save('results')
    